This adapted from a Coursera project, my contribution was around adapting the code to run in a modern AWS environment - primarily around IAM (access_key & secret_key.). Before doing this project I had no idea that this could be done - using a local Note Book to create a DynamoDB table.

In [12]:
import boto3
from botocore.exceptions import ClientError
from pprint import pprint

In [13]:
 # Define access key and secret key
access_key = '<your access key>'
secret_key = '<your secret key>'

# Create a session using the access key, secret key, region.
session = boto3.Session(
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name='us-east-2'  
)

# Initialize the DynamoDB resource using the session
dynamodb = session.resource('dynamodb')

In [14]:
table = dynamodb.create_table(
    TableName='Books02',
    KeySchema=[
        {
            'AttributeName': 'Author',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'Title',
            'KeyType': 'RANGE'  # Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'Author',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'Title',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)


In [15]:
print(table.table_status)

CREATING


In [16]:
import json
from decimal import Decimal
with open('books_data.json') as json_file:
    books_list = json.load(json_file,parse_float=Decimal)
#confirm   
print(books_list)

[{'Author': 'John Grisham', 'Title': 'The Rainmaker', 'Category': 'Suspense', 'Formats': {'Hardcover': 'J4SUKVGU', 'Paperback': 'D7YF4FCX'}}, {'Author': 'John Grisham', 'Title': 'The Firm', 'Category': 'Suspense', 'Formats': {'Hardcover': 'Q7QWE3U2', 'Paperback': 'ZVZAYY4F', 'Audiobook': 'DJ9KS9NM'}}, {'Author': 'James Patterson', 'Title': 'Along Came a Spider', 'Category': 'Suspense', 'Formats': {'Hardcover': 'C9NR6RJ7', 'Paperback': '37JVGDZG', 'Audiobook': '6348WX3U'}}, {'Author': 'Dr. Seuss', 'Title': 'Green Eggs and Ham', 'Category': 'Children', 'Formats': {'Hardcover': 'GVJZQ7JK', 'Paperback': 'A4TFUR98', 'Audiobook': 'XWMGHW96'}}, {'Author': 'William Shakespeare', 'Title': 'Hamlet', 'Category': 'Drama', 'Formats': {'Hardcover': 'GVJZQ7JK', 'Paperback': 'A4TFUR98', 'Audiobook': 'XWMGHW96'}}]


In [17]:
#'HTTPStatusCode': 200 response code will mean success
#load data
for book in books_list:
    print(book['Title'])
table.put_item(Item = book)

The Rainmaker
The Firm
Along Came a Spider
Green Eggs and Ham
Hamlet


{'ResponseMetadata': {'RequestId': 'DGLJ6SJTKMLTMR8RBVR128AVPRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 22 Jul 2024 16:30:20 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'DGLJ6SJTKMLTMR8RBVR128AVPRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [18]:
#insert item
table = dynamodb.Table('Books02')
response = table.put_item(
    Item={
        'Author': 'Mr Hopps',
        'Title': 'Hopps Guide to AWS',
        'Category': 'AWS DIY',
        'Formats': {
            'folio': 'xxx001',
            'Special Collectors': 'xxx002'
        }
    }
)

# Confirm insert
print(response)

{'ResponseMetadata': {'RequestId': 'S078T1JCC675BHJA4MDJCIKV13VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 22 Jul 2024 16:30:24 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'S078T1JCC675BHJA4MDJCIKV13VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


In [19]:
#read an item
# read an item
author = 'Mr Hopps'
title = 'Hopps Guide to AWS'
try:
    response = table.get_item(Key = {'Author':author, 'Title':title})
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    print(response['Item'])

{'Title': 'Hopps Guide to AWS', 'Formats': {'folio': 'xxx001', 'Special Collectors': 'xxx002'}, 'Author': 'Mr Hopps', 'Category': 'AWS DIY'}


In [20]:
# Update an item
category = 'Suspense'
response = table.update_item(
    Key={
        'Author': author,
        'Title': title
    },
    UpdateExpression='SET Category = :c',
    ExpressionAttributeValues={
        ':c': category
    },
    ReturnValues='UPDATED_NEW'
)

print(response)


{'Attributes': {'Category': 'Suspense'}, 'ResponseMetadata': {'RequestId': 'NJCADIQHG4JO1USP9VGV4D6CMFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 22 Jul 2024 16:30:46 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '44', 'connection': 'keep-alive', 'x-amzn-requestid': 'NJCADIQHG4JO1USP9VGV4D6CMFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '320359638'}, 'RetryAttempts': 0}}


In [21]:
# Query example
from boto3.dynamodb.conditions import Key

response = table.query(
    ProjectionExpression='Title, Category, #ft',
    ExpressionAttributeNames={'#ft': 'Format'},
    KeyConditionExpression=Key('Author').eq('JohnGrisham')
)

print(response)

{'Items': [], 'Count': 0, 'ScannedCount': 0, 'ResponseMetadata': {'RequestId': 'JH60MJ1JS3JG6RAFS939PO7TMFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Mon, 22 Jul 2024 16:30:49 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '39', 'connection': 'keep-alive', 'x-amzn-requestid': 'JH60MJ1JS3JG6RAFS939PO7TMFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3413411624'}, 'RetryAttempts': 0}}
